# regrid the dataset to a healpix grid

Since the default geographic rectilinear grid has non-uniform grid sizes and distances and is thus not suitable for this application, we have to transform the dataset to one with equal distances between the cell centers. The Healpix cells satisfy these conditions, and can be used for spatial convolutions when reshaped to a 2D array according to the "nested" cell numbering.

As with any interpolation, the linear interpolation supported by the `healpy` library is split into the computation of weights and the application of those weights to the data.

In [ ]:
import xarray as xr
from pangeo_fish.healpy import create_grid, HealpyRegridder

parameters

In [ ]:
nside: int = 4096  # healpix resolution
rot = {"lat": 0, "lon": 30}
original_path: str
regridded_path: str

scheduler_address: str = None

In [ ]:
root = "/home/jmagin/work/data/fish-intel"
input_path = f"{root}/diff/A18832-f1_e2500.zarr"
output_path = f"{root}/diff_healpix/A18832-f1_e2500_hp{nside}.zarr"

cluster

In [ ]:
from distributed import Client, LocalCluster

if scheduler_address is None:
    cluster = LocalCluster()
    client = cluster.get_client()
else:
    client = Client(scheduler_address)

client

## read the data

In [ ]:
ds = xr.open_dataset(input_path, engine="zarr", chunks={})
ds

## define the target grid

In [ ]:
%%time
grid = create_grid(nside=nside, rot=rot)
grid

## compute the weights

In [ ]:
%%time
regridder = HealpyRegridder(ds, grid)
regridder

## apply the weights

In [ ]:
%%time
regridded = regridder.regrid_ds(ds)
regridded

## compute and save to disk

In [ ]:
regridded.to_zarr(output_path, mode="w", consolidated=True, compute=True)

## visualize the results

In [ ]:
regridded_ = xr.open_dataset(output_path, engine="zarr", chunks={})
regridded_

In [ ]:
import hvplot.xarray
import cmocean
import holoviews as hv

hv.output(widget_location="bottom")

In [ ]:
regridded_["diff"].hvplot.quadmesh(
    x="longitude",
    y="latitude",
    rasterize=True,
    cmap="cmo.diff",
    coastline="10m",
    clim=(-5, 5),
).opts(frame_width=400)